    This code will:

    -Fetch the weather.gov page for the specified location
    -Extract current weather conditions
    -Extract the forecast data
    -Save both to separate CSV files with timestamps in the filenames
        also:
        What This Code Does:

    Creates organized directories for storing CSV files and HTML snapshots
    Scrapes the website 10 times with a 50-second pause between each scrape

    
    Extracts three types of data:

    -Current weather conditions (temperature, humidity, wind, etc.)
    -7-day forecast information
    -Location information


    Saves data to persistent files:

    -One CSV file for all current conditions (appends new data each time)
    -One CSV file for all forecast data (appends new data each time)
    -Separate HTML files for each scrape iteration (for reference)


    Provides informative console output during execution

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from datetime import datetime
import time
import os

def setup_directories():
    """Create directories for storing data if they don't exist"""
    directories = ['weather_data', 'weather_data/csv', 'weather_data/html']
    for directory in directories:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Created directory: {directory}")

def fetch_page(url):
    """Fetch the weather.gov web page"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        print(f"Successfully fetched {url}")
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def parse_current_conditions(html_content):
    """Extract current weather conditions"""
    if not html_content:
        return None
    
    soup = BeautifulSoup(html_content, 'html.parser')
    current_conditions = {}
    
    # Get current conditions section
    current_conditions_div = soup.find('div', id='current_conditions-summary')
    if current_conditions_div:
        # Get temperature
        temp_element = current_conditions_div.find('p', class_='myforecast-current-lrg')
        if temp_element:
            current_conditions['temperature'] = temp_element.text.strip()
        
        # Get weather condition text
        condition_element = current_conditions_div.find('p', class_='myforecast-current')
        if condition_element:
            current_conditions['condition'] = condition_element.text.strip()
        
        # Get additional details
        details_table = soup.find('div', id='current_conditions_detail')
        if details_table:
            rows = details_table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 2:
                    label = cells[0].text.strip().rstrip(':').lower().replace(' ', '_')
                    value = cells[1].text.strip()
                    current_conditions[label] = value
    
    # Add timestamp
    current_conditions['timestamp'] = datetime.now().isoformat()
    
    return current_conditions

def parse_forecast(html_content):
    """Extract forecast data"""
    if not html_content:
        return None
    
    soup = BeautifulSoup(html_content, 'html.parser')
    forecast_data = []
    
    # Find forecast section
    forecast_section = soup.find('div', id='seven-day-forecast-container')
    if forecast_section:
        # Get all forecast items
        forecast_items = forecast_section.find_all('div', class_='tombstone-container')
        
        for item in forecast_items:
            period_element = item.find('p', class_='period-name')
            desc_element = item.find('p', class_='short-desc')
            temp_element = item.find('p', class_='temp')
            img_element = item.find('img')
            
            forecast_item = {}
            
            if period_element:
                forecast_item['period'] = period_element.text.strip()
            
            if desc_element:
                forecast_item['description'] = desc_element.text.strip()
            
            if temp_element:
                forecast_item['temperature'] = temp_element.text.strip()
            
            if img_element and img_element.has_attr('alt'):
                forecast_item['condition'] = img_element['alt']
            
            # Add timestamp
            forecast_item['timestamp'] = datetime.now().isoformat()
            
            if forecast_item:  # Only add if we got some data
                forecast_data.append(forecast_item)
    
    return forecast_data

def parse_location_info(html_content):
    """Extract location information"""
    if not html_content:
        return None
    
    soup = BeautifulSoup(html_content, 'html.parser')
    location_info = {}
    
    # Find location section
    location_section = soup.find('div', class_='panel-heading')
    if location_section:
        location_text = location_section.text.strip()
        location_info['full_location'] = location_text
    
    return location_info

def save_to_csv(data, filename, mode='w'):
    """
    Save data to CSV file
    
    Args:
        data: Data to save (list of dicts or single dict)
        filename: File to save to
        mode: File open mode ('w' for write, 'a' for append)
    """
    if not data:
        print("No data to save")
        return
    
    # If data is a dictionary (current conditions), convert to list for CSV writing
    if isinstance(data, dict):
        data = [data]
    
    # Get field names from first item
    fieldnames = data[0].keys()
    
    # Check if file exists and mode is append
    file_exists = os.path.isfile(filename) and mode == 'a'
    
    # Write to CSV
    with open(filename, mode, newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Only write header if file is new or we're in write mode
        if not file_exists:
            writer.writeheader()
        
        writer.writerows(data)
    
    print(f"Data saved to {filename}")

def save_html(html_content, filename):
    """Save raw HTML content"""
    if not html_content:
        print("No HTML content to save")
        return
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"HTML saved to {filename}")

def scrape_and_save(url, current_csv, forecast_csv, html_file, iteration):
    """Perform one scrape cycle and save data"""
    print(f"\n--- Scrape Iteration #{iteration} ---")
    print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Fetch the page
    html_content = fetch_page(url)
    
    if html_content:
        # Save raw HTML
        save_html(html_content, html_file)
        
        # Get location info
        location_info = parse_location_info(html_content)
        if location_info:
            print(f"Location: {location_info.get('full_location', 'Unknown')}")
        
        # Get current conditions
        current_conditions = parse_current_conditions(html_content)
        if current_conditions:
            # Print summary
            temp = current_conditions.get('temperature', 'N/A')
            condition = current_conditions.get('condition', 'N/A')
            humidity = current_conditions.get('humidity', 'N/A')
            print(f"Current: {temp}, {condition}, Humidity: {humidity}")
            
            # Save current conditions to CSV (append mode)
            save_to_csv(current_conditions, current_csv, 'a')
        
        # Get forecast
        forecast_data = parse_forecast(html_content)
        if forecast_data:
            # Print first forecast period
            if forecast_data:
                first_period = forecast_data[0].get('period', 'N/A')
                first_temp = forecast_data[0].get('temperature', 'N/A')
                print(f"Forecast for {first_period}: {first_temp}")
            
            # Save forecast to CSV (append mode)
            save_to_csv(forecast_data, forecast_csv, 'a')
    
    return html_content is not None  # Return success status

def main():
    """Main entry point"""
    # Create directories
    setup_directories()
    
    # Weather.gov URL
    url = "https://forecast.weather.gov/MapClick.php?lat=47.449&lon=-122.3093"
    
    # Set up file paths with timestamps
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    current_csv = f"weather_data/csv/current_weather_{timestamp}.csv"
    forecast_csv = f"weather_data/csv/forecast_{timestamp}.csv"
    
    # Number of iterations and interval
    iterations = 10
    interval_seconds = 50
    
    print(f"Starting weather scraper for {iterations} iterations")
    print(f"Interval between scrapes: {interval_seconds} seconds")
    print(f"Current conditions will be saved to: {current_csv}")
    print(f"Forecast data will be saved to: {forecast_csv}")
    print(f"Raw HTML will be saved to weather_data/html/")
    
    # Run scraper for specified number of iterations
    successful_scrapes = 0
    for i in range(1, iterations + 1):
        # Create HTML filename with iteration number
        html_file = f"weather_data/html/weather_{timestamp}_iteration_{i}.html"
        
        # Perform scrape
        success = scrape_and_save(url, current_csv, forecast_csv, html_file, i)
        if success:
            successful_scrapes += 1
        
        # Sleep between iterations (except for the last one)
        if i < iterations:
            print(f"Sleeping for {interval_seconds} seconds...")
            time.sleep(interval_seconds)
    
    print(f"\nScraping completed: {successful_scrapes} successful out of {iterations} attempts")
    print(f"Current conditions saved to: {current_csv}")
    print(f"Forecast data saved to: {forecast_csv}")

if __name__ == "__main__":
    main()

Created directory: weather_data
Created directory: weather_data/csv
Created directory: weather_data/html
Starting weather scraper for 10 iterations
Interval between scrapes: 50 seconds
Current conditions will be saved to: weather_data/csv/current_weather_20250308_050643.csv
Forecast data will be saved to: weather_data/csv/forecast_20250308_050643.csv
Raw HTML will be saved to weather_data/html/

--- Scrape Iteration #1 ---
Time: 2025-03-08 05:06:43
Successfully fetched https://forecast.weather.gov/MapClick.php?lat=47.449&lon=-122.3093
HTML saved to weather_data/html/weather_20250308_050643_iteration_1.html
Location: Current conditions at
Seattle, Seattle-Tacoma International Airport (KSEA)
Lat: 47.44472°NLon: 122.31361°WElev: 427.0ft.
Current: 48°F, Mostly Cloudy, Humidity: 60%
Data saved to weather_data/csv/current_weather_20250308_050643.csv
Forecast for Tonight: Low: 41 °F
Data saved to weather_data/csv/forecast_20250308_050643.csv
Sleeping for 50 seconds...

--- Scrape Iteration #2